In [1]:
# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

2.12.1
# GPUs Available:  1


In [2]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)


data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'tot_residual_flow', 
       'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[['leak_1', 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[['leak_1', 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[['leak_1', 'leak_2']]

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

y1_train['x2'] = y1_train['x2'].replace(np.nan, -5)
y1_train['y2'] = y1_train['y2'].replace(np.nan, -5)

y1_test['x2'] = y1_test['x2'].replace(np.nan, -5)
y1_test['y2'] = y1_test['y2'].replace(np.nan, -5)

y1_val['x2'] = y1_val['x2'].replace(np.nan, -5)
y1_val['y2'] = y1_val['y2'].replace(np.nan, -5)
# Not sure if 0 is good enough or try generating a random number

# scaler_coords2 = StandardScaler()
# y2_train = scaler_coords2.fit_transform(y2_train)
# y2_test = scaler_coords2.fit_transform(y2_test)
# y2_val = scaler_coords2.transform(y2_val)

# y2_train = pd.DataFrame(y2_train, columns=y2_columns)
# y2_test = pd.DataFrame(y2_test, columns=y2_columns)
# y2_val = pd.DataFrame(y2_val, columns=y2_columns)

y2_train = y2_train.reset_index().drop(columns='sample_number')
y2_val = y2_val.reset_index().drop(columns='sample_number')
y2_test = y2_test.reset_index().drop(columns='sample_number')

y_train = pd.concat([y1_train, y2_train], axis=1)
y_test = pd.concat([y1_test, y2_test], axis=1)
y_val = pd.concat([y1_val, y2_val], axis=1)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

In [5]:
# Fitting a simple linear regression model
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [6]:
# converting the predictions to certainity of 0 and 1 (May not be a good idea)
y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1

y_pred = y_pred.transpose()

mse_x1 = mean_squared_error(y_test['x1'], y_pred[0], squared=True)
mse_x2 = mean_squared_error(y_test['x2'], y_pred[2], squared=True)

mse_y1 = mean_squared_error(y_test['y1'], y_pred[1], squared=True)
mse_y2 = mean_squared_error(y_test['y2'], y_pred[3], squared=True)

mse_leak1 = mean_squared_error(y_test['leak_1'], y_pred[4], squared=True)
mse_leak2 = mean_squared_error(y_test['leak_2'], y_pred[5], squared=True)

mse = [mse_x1, mse_x2, mse_y1, mse_y2, mse_leak1, mse_leak2]

In [7]:
y_pred = pd.DataFrame(y_pred.transpose(), columns=y_train.columns)

pd.concat([y_pred, y_test], axis=1)

,x1,y1,x2,y2,leak_1,leak_2,x1,y1,x2,y2,leak_1,leak_2
0,1.063788,0.367663,2.070305,1.543146,1.0,1.0,-0.232617,-0.472725,1.831046,1.164705,1,1
1,-0.405377,-1.364914,0.372441,-0.071279,1.0,1.0,0.574618,-1.002915,-0.953740,-0.988391,1,1
2,-0.970896,0.163662,-0.112895,0.526664,1.0,1.0,-1.385810,0.410927,-0.005728,0.447007,1,1
3,-0.486326,0.372816,0.068367,0.637381,1.0,1.0,0.343979,0.941118,-0.894489,-0.091267,1,1
4,0.067027,-0.047254,-0.017044,-0.179627,1.0,1.0,1.151215,0.057466,-1.131492,-0.270692,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.897223,-1.036307,-0.224278,-1.336261,1.0,1.0,0.459299,-1.179646,1.060786,-1.167815,1,1
132,-0.700991,-0.239518,-5.098910,-4.832878,1.0,0.0,-1.328151,-0.295994,-5.000000,-5.000000,1,0
133,1.037478,0.569600,-2.790430,-3.193737,1.0,0.0,1.554832,0.764387,-5.000000,-5.000000,1,0
134,0.776857,0.354502,-4.613408,-4.922989,1.0,0.0,1.324194,0.410927,-5.000000,-5.000000,1,0


In [8]:
mse

[0.30438715505404185,
 0.6401107158640298,
 0.28556211416781063,
 0.8163172969896335,
 0.0,
 0.0]

In [9]:
#Define the model using model builder from keras tuner
def model_builder_single(hp):
    model = keras.Sequential()

    # Choose an optimal value between 32-512
    for i in range(hp.Int("num_layers", 1, 15)):
        l1_weight = hp.Choice('l1_weight', values=[0.0, 1e-1, 1e-2, 1e-3])
        l2_weight = hp.Choice('l2_weight', values=[0.0, 1e-1, 1e-2, 1e-3])
        kernel_regularizer=keras.regularizers.L1L2(l1 = l1_weight, l2 = l2_weight)
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
                kernel_initializer='he_uniform',
                kernel_regularizer=kernel_regularizer
            )
        )
    model.add(keras.layers.Dense(units=6, activation= "linear", kernel_initializer='he_uniform'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

    # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    #             loss="mse",  metrics='mae')
    
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate),
                loss="mse",
                metrics='mae')

    return model


#search for the best hyperparameters and train the standard model with original training data
def hyper_model(X_train,Y_train, X_val, y_val, epoch, factor, augmentation, residual_subtract, 
                blind_flip, tot_flow, res_flow, tot_resflow):
    folder_name = str('keras_hyperparameter_aug_'+ str(augmentation)+
                      "_res_"+ str(residual_subtract)+
                      "_blind_"+ str(blind_flip)+
                      "tot_flow"+ str(tot_flow)+
                      "res_flow"+ str(res_flow)+
                      "tot_resflow"+ str(tot_resflow)
                    )
    tuner = kt.Hyperband(model_builder_single,
                         objective='val_loss',
                         max_epochs=epoch,
                         factor=factor,
                         hyperband_iterations = 1,
                        # Integer, at least 1, the number of times to iterate over the full Hyperband algorithm. One iteration will 
                        # run approximately max_epochs * (math.log(max_epochs, factor) ** 2) cumulative epochs across all trials. It is 
                        # recommended to set this to as high a value as is within your resource budget. Defaults to 1.
                         directory="../../tensorflow_log_files/studienarbeit/",
                         seed=0,    
                         project_name=str(folder_name))

    tuner.search_space_summary()
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    tuner.search(X_train, Y_train, epochs=epoch, validation_data = (X_val, y_val), callbacks=[stop_early, 
                                                                                            #   keras.callbacks.TensorBoard("../tensorflow_log_files/studienarbeit/tb_logs"+str(folder_name))
                                                                                              ])
    #tuner.search(X_train, Y_train, epochs=50, validation_data=(X_test,Y_test), callbacks=[stop_early])
    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    model = tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, Y_train, epochs=epoch, validation_data = (X_val, y_val), shuffle= True)

    print(f"""
    The hyperparameter search is complete. The optimal learning rate for the optimizer
    is {model.optimizer.lr.numpy()}.
    """)

    return best_hps, model, tuner, history

In [10]:
best_hps, best_model, tuner, history = hyper_model(X_train,y_train, X_val, y_val,
                                                        cfg['experiment']['EPOCH'], cfg['experiment']['factor'], False, 
                                                        False, False, False, False, 'False_two_leak_1loss')

INFO:tensorflow:Reloading Tuner from ../../tensorflow_log_files/studienarbeit/keras_hyperparameter_aug_False_res_False_blind_Falsetot_flowFalseres_flowFalsetot_resflowFalse_two_leak_1loss/tuner0.json
Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 15, 'step': 1, 'sampling': 'linear'}
l1_weight (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.01, 0.001], 'ordered': True}
l2_weight (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.01, 0.001], 'ordered': True}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.1, 'step': None, 'sampling': 'log'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_va

In [11]:
y_pred = best_model.predict(X_test)

5/5 [==============================] - 0s 2ms/step


In [12]:
# converting the predictions to certainity of 0 and 1 (May not be a good idea)
y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1

y_pred = y_pred.transpose()

mse_x1 = mean_squared_error(y_test['x1'], y_pred[0], squared=True)
mse_x2 = mean_squared_error(y_test['x2'], y_pred[2], squared=True)

mse_y1 = mean_squared_error(y_test['y1'], y_pred[1], squared=True)
mse_y2 = mean_squared_error(y_test['y2'], y_pred[3], squared=True)

mse_leak1 = mean_squared_error(y_test['leak_1'], y_pred[4], squared=True)
mse_leak2 = mean_squared_error(y_test['leak_2'], y_pred[5], squared=True)

mse = [mse_x1, mse_x2, mse_y1, mse_y2, mse_leak1, mse_leak2]

In [13]:
y_pred = pd.DataFrame(y_pred.transpose(), columns=y_train.columns)

pd.concat([y_pred, y_test], axis=1)

,x1,y1,x2,y2,leak_1,leak_2,x1,y1,x2,y2,leak_1,leak_2
0,-0.253605,-0.445571,1.227548,0.834547,1.0,1.0,-0.232617,-0.472725,1.831046,1.164705,1,1
1,-0.007252,-0.860756,-0.611063,-0.977354,1.0,1.0,0.574618,-1.002915,-0.953740,-0.988391,1,1
2,-0.424406,0.238467,-0.353680,0.175441,1.0,1.0,-1.385810,0.410927,-0.005728,0.447007,1,1
3,-0.442701,0.252443,-0.410114,0.222157,1.0,1.0,0.343979,0.941118,-0.894489,-0.091267,1,1
4,-0.390945,-0.309759,0.759396,0.649354,1.0,1.0,1.151215,0.057466,-1.131492,-0.270692,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.755948,-1.092538,0.708057,-1.180654,1.0,1.0,0.459299,-1.179646,1.060786,-1.167815,1,1
132,-0.995168,-0.160978,-5.054129,-5.073519,1.0,0.0,-1.328151,-0.295994,-5.000000,-5.000000,1,0
133,1.306913,1.234398,-4.890694,-4.865629,1.0,0.0,1.554832,0.764387,-5.000000,-5.000000,1,0
134,1.353195,0.600954,-4.962316,-4.959120,1.0,0.0,1.324194,0.410927,-5.000000,-5.000000,1,0


In [14]:
mse

[0.33533855384610567,
 0.31527507970236945,
 0.22006477042068587,
 0.244060215425545,
 0.0,
 0.0]